Note: excluded:
- data_pKA = read_sdf(pKA_path, 'pKa')
- data_RBiodeg = read_sdf(RBiodeg_path, 'RBiodeg') binary
- data_CoMPARA = read_sdf(CoMPARA_path, 'CoMPARA'), mixed 
- data_CATMOS = read_sdf(CATMOS_path, 'CATMOS') mixed
- data_CERAPP = read_sdf(CERAPP_path, 'CERAPP') mixed


In [ ]:
from utils_data import load_data,scale_props,get_graphs
from utils_plotting import plot_property_histograms
from sklearn.model_selection import train_test_split
from utils_model import train_and_validate_multi,get_preds_per_task,preds_and_ys_to_df
from torch_geometric.loader import DataLoader
from torch_geometric.nn.models import AttentiveFP
import torch
import pandas as pd

: 

In [ ]:
df_combined = load_data()
df_combined

: 

In [ ]:
plot_property_histograms(df_combined)

: 

In [ ]:
df_scaled,scaler = scale_props(df_combined)
#save the scaler
scaler.save('scaler.pkl')
plot_property_histograms(df_scaled)

: 

In [ ]:
#split check: if we do a random split 80/20, do we also get a 80/20 split for each property?
train, test = train_test_split(df_scaled, test_size=0.2, random_state=42)
for col in df_scaled.columns[1:]:
    print(f'{col}: {test[col].count()/df_scaled[col].count()}')

: 

In [ ]:
#check if for each property the distribution is similar in train and test
from scipy.stats import mannwhitneyu
for col in df_scaled.columns[1:]:
    train_tmp = train[col].dropna()
    test_tmp = test[col].dropna()
    print(f'{col}: {mannwhitneyu(train_tmp,test_tmp)}')
import seaborn as sns
import matplotlib.pyplot as plt
for col in df_scaled.columns[1:]:
    plt.boxplot(x=[train[col].dropna(),test[col].dropna()],labels=['train','test'])
    plt.show()

: 

In [ ]:
#count duplicated smiles
print(f'duplicated smiles: {df_scaled.SMILES.duplicated().sum()}')
#lets save these splits
train.to_csv('train.csv',index=False)
test.to_csv('test.csv',index=False)
print(len(train),len(test))


: 

In [ ]:
# train_graphs_no_charge_scaled = get_graphs(train,dash_charges=False,scaled =True,save_graphs = True)

: 

In [ ]:
train_graphs_DASH_charge_scaled = get_graphs(train,dash_charges=True,scaled =True,save_graphs = True)

: 

In [ ]:
outputs = 14
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_data, val_data = train_test_split(train_graphs_DASH_charge_scaled, test_size=0.2, random_state=2000)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64, shuffle=False)
model= AttentiveFP(in_channels=24, hidden_channels=200, out_channels=outputs, #note that using the DASH graphs will increase the amount of node feauters (input channels for model) from 23 to 24
                            edge_dim=11, num_layers=4, num_timesteps=2,
                            dropout=0.0).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=10**-3,
                        weight_decay=10**-4)
print(len(train_data), len(val_data))
train_and_validate_multi(model, train_loader, val_loader, optimizer, num_epochs=100, outputs=outputs, verbose=True,props_to_train=['LogVP','LogP'])

: 

In [ ]:
preds, ys = get_preds_per_task(model,val_loader, outputs)

: 

In [ ]:
#make df from preds
model= AttentiveFP(in_channels=24, hidden_channels=200, out_channels=outputs,
                        edge_dim=11, num_layers=4, num_timesteps=2,
                        dropout=0.0).to(device)
model.load_state_dict(torch.load('test_model.pt'))
df_preds,df_ys = preds_and_ys_to_df(preds,ys,ref_df = df_combined,scaler = scaler,props_to_train=['LogVP','LogP'])

: 

In [ ]:
#plot
from utils_plotting import plot_scatters
plot_scatters(df_preds,df_ys)

: 

To do:
1. Add the different endpoints - check
2. Normalization of values
3. Weighting of the tasks
4. Hyperparameter optimization
5. Benchmarking datasets? E.g. SAMPL7 for logP
6. Quadruple check refs - check
7. Think about datasplitting: same splits as OPERA? possible? otherwise, just cross-val? or random splits (multiple)
8. Different random weight initializations of the model


split: 80/20 5x

HP OPT: https://docs.ray.io/en/latest/tune/api/doc/ray.tune.search.bayesopt.BayesOptSearch.html
Repeated K-fold: https://greglandrum.github.io/rdkit-blog/posts/2023-08-13-xval-variability1.html